# Lightweight networks and MobileNet

복잡한 네트워크는 훈련과 빠른 추론을 위해 GPU와 같은 상당한 계산 리소스가 필요하다는 것을 확인했습니다. 그러나 대부분의 경우 파라미터의 수가 훨씬 적은 모델은 여전히 합리적으로 잘 수행되도록 훈련될 수 있으며, 모델 복잡성이 증가하면 일반적으로 모델 성능이 약간 증가하게 됩니다.

MNIST 숫자 분류를 학습 할 때 모듈 시작 부분에서 이를 관찰했습니다. 단순한 Dense Layer 모델의 정확도는 강력한 CNN보다 크게 나쁘지는 않았습니다. 분류기에서 CNN Layer 수 또는 뉴런 수를 늘리면 몇 퍼센트 증가한 정확도를 얻을 수 있었습니다.

이를 통해 우리는 더 빠른 모델을 훈련시키기 위해 경량 네트워크 아키텍처를 훈련할 수 있다는 생각으로 이어집니다. 특히 모바일 장치에서 모델을 실행할 수 있기를 원할 때 이러한 부분은 중요합니다.

이 모듈은 이전 챕터에서 다운로드 한 Cats and Dogs 데이터셋에 의존합니다. 먼저 데이터셋을 사용할 수 있는지 확인합니다.

In [1]:
import torch
import torch.nn as nn
import torchvision
import matplotlib.pyplot as plt
from torchinfo import summary
import os

from pytorchcv import train, display_dataset, train_long, load_cats_dogs_dataset, validate, common_transform

In [2]:
if not os.path.exists('data/kagglecatsanddogs_3367a.zip'):
    !wget -P data -q https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_3367a.zip

dataset, train_loader, test_loader = load_cats_dogs_dataset()

/anaconda/envs/py37_pytorch/lib/python3.7/site-packages/PIL/TiffImagePlugin.py:770: UserWarning: Possibly corrupt EXIF data.  Expecting to read 32 bytes but only got 0. Skipping tag 270
  " Skipping tag %s" % (size, len(data), tag)
/anaconda/envs/py37_pytorch/lib/python3.7/site-packages/PIL/TiffImagePlugin.py:770: UserWarning: Possibly corrupt EXIF data.  Expecting to read 5 bytes but only got 0. Skipping tag 271
  " Skipping tag %s" % (size, len(data), tag)
/anaconda/envs/py37_pytorch/lib/python3.7/site-packages/PIL/TiffImagePlugin.py:770: UserWarning: Possibly corrupt EXIF data.  Expecting to read 8 bytes but only got 0. Skipping tag 272
  " Skipping tag %s" % (size, len(data), tag)
/anaconda/envs/py37_pytorch/lib/python3.7/site-packages/PIL/TiffImagePlugin.py:770: UserWarning: Possibly corrupt EXIF data.  Expecting to read 8 bytes but only got 0. Skipping tag 282
  " Skipping tag %s" % (size, len(data), tag)
/anaconda/envs/py37_pytorch/lib/python3.7/site-packages/PIL/TiffImagePlugin

## MobileNet

이전 챕터에서 이미지 분류를 위한 **ResNet** 아키텍처를 살펴 보았습니다. ResNet의 더 가벼운 버전의 모델은 **MobileNet**으로, 이른바 *Inverted Residual Blocks*를 사용합니다. 사전 훈련된 Mobile Net을 불러와 작동 방식을 살펴보겠습니다.

In [3]:
model = torch.hub.load('pytorch/vision:v0.6.0', 'mobilenet_v2', pretrained=True)
model.eval()
print(model)

Using cache found in /home/vmuser/.cache/torch/hub/pytorch_vision_v0.6.0


MobileNetV2(
  (features): Sequential(
    (0): ConvBNReLU(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): ConvBNReLU(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): ConvBNReLU(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=Tr

이 모델을 데이터셋에 적용하여 제대로 작동하는지 확인해 보겠습니다.

In [4]:
sample_image = dataset[0][0].unsqueeze(0)
res = model(sample_image)
print(res[0].argmax())

tensor(281)


**Exercise:** MobileNet과 ResNet의 파라미터 수를 비교합니다.

## Using MobileNet for transfer learning

이제 이전 챕터와 동일한 전이 학습 프로세스를 수행하되, MobileNet을 사용해 보겠습니다. 먼저 모델의 모든 파라미터를 고정합니다.

In [5]:
for x in model.parameters():
    x.requires_grad = False

그런 다음 최종 분류기를 교체하고, 모델을 기본 훈련 장치(GPU 또는 CPU)로 전송합니다.

In [7]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.classifier = nn.Linear(1280,2)
model = model.to(device)
summary(model,input_size=(1,3,244,244))

Layer (type:depth-idx)                   Output Shape              Param #
├─Sequential: 1-1                        [1, 1280, 8, 8]           --
|    └─ConvBNReLU: 2-1                   [1, 32, 122, 122]         --
|    |    └─Conv2d: 3-1                  [1, 32, 122, 122]         (864)
|    |    └─BatchNorm2d: 3-2             [1, 32, 122, 122]         (64)
|    |    └─ReLU6: 3-3                   [1, 32, 122, 122]         --
|    └─InvertedResidual: 2-2             [1, 16, 122, 122]         --
|    |    └─Sequential: 3-4              [1, 16, 122, 122]         (896)
|    └─InvertedResidual: 2-3             [1, 24, 61, 61]           --
|    |    └─Sequential: 3-5              [1, 24, 61, 61]           (5,136)
|    └─InvertedResidual: 2-4             [1, 24, 61, 61]           --
|    |    └─Sequential: 3-6              [1, 24, 61, 61]           (8,832)
|    └─InvertedResidual: 2-5             [1, 32, 31, 31]           --
|    |    └─Sequential: 3-7              [1, 32, 31, 31]           

이제 실제 학습을 해봅시다.

In [8]:
train_long(model,train_loader,test_loader,loss_fn=torch.nn.CrossEntropyLoss(),epochs=1,print_freq=90)

Epoch 0, minibatch 0: train acc = 0.5, train loss = 0.02309325896203518
Epoch 0, minibatch 90: train acc = 0.9443681318681318, train loss = 0.006317565729329874
Epoch 0, minibatch 180: train acc = 0.9488950276243094, train loss = 0.00590015182178982
Epoch 0, minibatch 270: train acc = 0.9492619926199262, train loss = 0.006072205810969167
Epoch 0, minibatch 360: train acc = 0.9500519390581718, train loss = 0.00641324315374908
Epoch 0, minibatch 450: train acc = 0.9494872505543237, train loss = 0.006945275943189397
Epoch 0, minibatch 540: train acc = 0.9521141404805915, train loss = 0.0067323536617257896
Epoch 0 done, validation acc = 0.98245, validation loss = 0.002347727584838867


## Takeaway


MobileNet은 VGG-16과 거의 동일한 정확도를 나타내며 full-scale ResNet보다 약간 낮게 나타납니다.

MobileNet 또는 ResNet-18과 같은 소형 모델의 주요 장점은 모바일 장치에서 사용할 수 있다는 것입니다. [예시1](https://pytorch.org/mobile/android/)은 Android 기기에서 ResNet-18을 사용하는 공식 예이며 [예시2](https://heartbeat.fritz.ai/pytorch-mobile-image-classification-on-android-5c0cfb774c5b)는 MobileNet을 사용하는 유사한 예입니다.